In [87]:
import numpy as np
import random

## Simple Test Problem

In [88]:
# CITIES = [
#     "Rome",
#     "Milan",
#     "Naples",
#     "Turin",
#     "Palermo",
#     "Genoa",
#     "Bologna",
#     "Florence",
#     "Bari",
#     "Catania",
#     "Venice",
#     "Verona",
#     "Messina",
#     "Padua",
#     "Trieste",
#     "Taranto",
#     "Brescia",
#     "Prato",
#     "Parma",
#     "Modena",
# ]
# test_problem = np.load("lab2/test_problem.npy")

## Common tests

In [89]:
# problem = np.load("lab2/problem_r2_1000.npy")
problem = np.load("lab2/problem_r1_100.npy")
# problem = test_problem

In [90]:
# Negative values?
np.any(problem < 0)

np.False_

In [91]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

True

In [92]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [93]:
# # Triangular inequality
# all(
#     problem[x, y] <= problem[x, z] + problem[z, y]
#     for x, y, z in list(combinations(range(problem.shape[0]), 3))
# )

# Solution

In [94]:
from icecream import ic
import matplotlib.pyplot as plt
from dataclasses import dataclass
import functools
from numba import njit
import numba as nb

## Params & Init

In [95]:
@dataclass
class SolverParams:
    pop_size: int = 50
    churn_rate: float = 0.5
    mutate_children_rate: float = 1
    wipeout_every: int = 5000
    survivors_num: int = 3
    children_per_pair: int = 4
    num_islands: int = 1
    migration_every: int = 1000
    migration_rate: float = 0.2


PARAMS = SolverParams()
ITEMS_NUM: int = problem.shape[0]


rng = np.random.default_rng(42)


def init_pop():
    # Randomly initialize the population with random permutations
    return rng.permuted(
        np.tile(np.arange(ITEMS_NUM, dtype=np.int64), PARAMS.pop_size).reshape(
            (PARAMS.pop_size, ITEMS_NUM)
        ),
        axis=1,
    )


population = init_pop()
ic(population)

ic| population: array([[59, 21, 56, ..., 65, 77,  8],
                       [34, 26, 42, ..., 22, 78, 44],
                       [74, 69, 61, ...,  1, 32, 60],
                       ...,
                       [10, 75, 37, ..., 35, 50, 84],
                       [58, 89,  6, ..., 62, 28, 36],
                       [96, 67, 72, ..., 55, 76, 40]], shape=(50, 100))


array([[59, 21, 56, ..., 65, 77,  8],
       [34, 26, 42, ..., 22, 78, 44],
       [74, 69, 61, ...,  1, 32, 60],
       ...,
       [10, 75, 37, ..., 35, 50, 84],
       [58, 89,  6, ..., 62, 28, 36],
       [96, 67, 72, ..., 55, 76, 40]], shape=(50, 100))

In [96]:
@njit(nb.types.float64(nb.types.Array(nb.int64, 1, "A")))
def calc_fitness(sol):
    return problem.reshape(-1)[sol * problem.shape[1] + np.roll(sol, -1)].sum()


@njit(
    nb.types.Array(nb.float64, 1, "C")(nb.types.Array(nb.int64, 2, "A")), parallel=True
)
def calc_pop_fitness(pop):
    # Calculates the fitness of individuals

    fitness = np.zeros(pop.shape[0], dtype=np.float64)
    for i in nb.prange(pop.shape[0]):
        fitness[i] = calc_fitness(pop[i])
    # costs = problem[pop, np.roll(pop, nb.int64(-1), nb.int64(1))]
    # print(costs.shape)
    # fitness = costs.sum(axis=1)
    return fitness

In [97]:
# %timeit calc_pop_fitness(init_pop())  # Test call

## Mutations

In [98]:
def mutate_solutions_swap(sols):
    # if len(sols.shape) == 1:
    #     sols = sols.reshape(1, -1)

    N = sols.shape[0]
    # For each individual, we swap two items
    swap_idxs = rng.integers(0, ITEMS_NUM, (N, 2))

    # out = sols.copy()

    tmp = sols[np.arange(N), swap_idxs[:, 0]]
    sols[np.arange(N), swap_idxs[:, 0]] = sols[np.arange(N), swap_idxs[:, 1]]
    sols[np.arange(N), swap_idxs[:, 1]] = tmp

    return sols


def mutate_solutions_swap_mul(sols, error_p=0.03):
    # if len(sols.shape) == 1:
    #     sols = sols.reshape(1, -1)

    N = sols.shape[0]
    # For each individual, we swap two items

    swaps_num = rng.binomial(ITEMS_NUM, error_p, size=N)

    # out = sols.copy()

    # ic(swaps_num)
    while swaps_num.max() > 0:
        (to_swap,) = np.where(swaps_num > 0)
        # ic(to_swap)
        swaps_num[to_swap] -= 1
        swap_idxs = rng.integers(0, ITEMS_NUM, (len(to_swap), 2))
        tmp = sols[to_swap, swap_idxs[:, 0]]
        sols[to_swap, swap_idxs[:, 0]] = sols[to_swap, swap_idxs[:, 1]]
        sols[to_swap, swap_idxs[:, 1]] = tmp

    return sols


def mutate_solutions_rev(sols):
    # if len(sols.shape) == 1:
    #     sols = sols.reshape(1, -1)

    N = sols.shape[0]
    # For each individual, we swap two items
    swap_idxs = rng.integers(0, ITEMS_NUM, (N, 2))
    swap_idxs = np.sort(swap_idxs, axis=1)
    # ic(swap_idxs)

    # out = sols.copy()

    for i in range(N):
        sols[i, swap_idxs[i, 0] : swap_idxs[i, 1]] = np.flip(
            sols[i, swap_idxs[i, 0] : swap_idxs[i, 1]]
        )

    return sols


def mutate_solutions_rev_v2(sols, len_p=0.1):
    # if len(sols.shape) == 1:
    #     sols = sols.reshape(1, -1)

    N = sols.shape[0]
    # For each individual, we swap two items
    swap_idxs_a = rng.integers(0, ITEMS_NUM, (N))
    swap_idxs_b = swap_idxs_a + rng.binomial(ITEMS_NUM, len_p, (N))
    # ic(swap_idxs)

    # out = sols.copy()

    for i in range(N):
        sols[i, swap_idxs_a[i] : swap_idxs_b[i]] = np.flip(
            sols[i, swap_idxs_a[i] : swap_idxs_b[i]]
        )

    return sols


def mutate_solutions_shift(sols):
    # if len(sols.shape) == 1:
    #     sols = sols.reshape(1, -1)

    N = sols.shape[0]
    # For each individual, we swap two items
    swap_idxs = rng.integers(0, ITEMS_NUM, (N, 2))
    swap_idxs = np.sort(swap_idxs, axis=1)
    # ic(swap_idxs)

    # out = sols.copy()

    for i in range(N):
        sols[i, swap_idxs[i, 0] : swap_idxs[i, 1]] = np.roll(
            sols[i, swap_idxs[i, 0] : swap_idxs[i, 1]], 1
        )

    return sols


def mutate_solutions_shift_v2(sols, len_p=0.1, amount_p=0.04):
    # if len(sols.shape) == 1:
    #     sols = sols.reshape(1, -1)

    N = sols.shape[0]
    # For each individual, we swap two items
    shift_a_idxs = rng.integers(0, ITEMS_NUM, (N))
    shift_len_idxs = rng.binomial(ITEMS_NUM, len_p, (N))
    shift_amount_idxs = rng.binomial(ITEMS_NUM, amount_p, (N))
    # ic(swap_idxs)

    # out = sols.copy()

    for i in range(N):
        sols[
            i,
            shift_a_idxs[i] : ((shift_a_idxs[i] + shift_len_idxs[i]) % ITEMS_NUM),
        ] = np.roll(
            sols[
                i,
                shift_a_idxs[i] : ((shift_a_idxs[i] + shift_len_idxs[i]) % ITEMS_NUM),
            ],
            shift_amount_idxs[i],
        )

    return sols


def mutate_partial_reset(sols):
    N = sols.shape[0]

    rands = rng.integers(0, ITEMS_NUM, (N, 2))

    for i in range(N):
        sols[i] = np.roll(sols[i], rands[i, 0])
        rng.shuffle(sols[i, : rands[i, 1]])

    return sols


# def mutate_inv(sols, amount=0.1):
#     N = sols.shape[0]

#     rands = rng.integers(0, ITEMS_NUM, (N, int(ITEMS_NUM * amount)))

#     sols[np.arange(N), rands] = np.flip(sols[np.arange(N), rands], axis=1)

#     return sols


def mutate_random(sols):
    # if len(sols.shape) == 1:
    #     sols = sols.reshape(1, -1)

    N = sols.shape[0]

    methods = [
        mutate_solutions_swap,
        mutate_solutions_shift_v2,
        # mutate_solutions_rev,
        mutate_solutions_rev_v2,
        # mutate_solutions_swap_mul,
        # mutate_partial_reset,
        # mutate_solutions_shift,
        # mutate_solutions_shift_v2,
    ]

    choices = rng.integers(0, len(methods), (N))
    # ic(choices)

    out = np.empty((N, ITEMS_NUM), dtype=int)

    for i in range(len(methods)):
        out[choices == i] = methods[i](sols[choices == i])

    return out

## Crossover

### Join Solutions

In [99]:
@njit
def cycle(i, sols_a, sols_b, roll_amounts, first_parent, flip_second):
    b = sols_a.shape[0]

    sol_a = np.roll(sols_a[i % b], roll_amounts[i, 0])
    sol_b = np.roll(sols_b[i % b], roll_amounts[i, 1])

    cursor = roll_amounts[i, 2]
    sol_a, sol_b = (sol_a, sol_b) if first_parent[i] == 0 else (sol_b, sol_a)
    if flip_second[i]:
        sol_b = np.flip(sol_b)

    out = np.empty((ITEMS_NUM), dtype=np.int64)

    out[:cursor] = sol_a[:cursor]
    missing = np.ones((ITEMS_NUM), dtype=np.bool)
    missing[sol_a[:cursor]] = 0

    for j in range(ITEMS_NUM):
        if missing[sol_b[j]]:
            out[cursor] = sol_b[j]
            cursor += 1

    return out


@njit(parallel=True)
def cycles(
    sols_a, sols_b, roll_amounts, first_parent, flip_second, out, children_per_pair
):
    N = sols_a.shape[0]

    for i in nb.prange(N):
        best_sol = cycle(i, sols_a, sols_b, roll_amounts, first_parent, flip_second)
        best_fitness = calc_fitness(best_sol)
        for _ in nb.prange(children_per_pair - 1):
            sol = cycle(i, sols_a, sols_b, roll_amounts, first_parent, flip_second)
            fitness = calc_fitness(sol)
            if fitness < best_fitness:
                best_fitness = fitness
                best_sol = sol

        out[i, :] = best_sol


# @njit
def join_solutions(sols_a, sols_b, children_per_pair):
    N = sols_a.shape[0]

    # rng = np.random.default_rng(42)
    roll_amounts = np.random.randint(0, ITEMS_NUM, (N, 3))
    first_parent = np.random.randint(0, 2, size=(N))
    flip_second = np.random.randint(0, 2, size=(N))

    out = np.full((N, ITEMS_NUM), -1, dtype=np.int64)

    cycles(
        sols_a,
        sols_b,
        roll_amounts,
        first_parent,
        flip_second,
        out,
        children_per_pair,
    )

    return out

In [100]:
join_solutions(init_pop()[:2], init_pop()[2:4], 2)

array([[14, 41, 34, 70, 84, 42, 39, 54,  8, 43, 22, 77, 92, 76, 52, 62,
        44, 99, 75, 66, 91,  3,  7, 50, 28, 57, 46, 73, 97, 94, 36, 17,
        11, 82, 47, 79, 55, 18, 69,  1, 58, 98, 81, 33,  4, 86, 72, 30,
        13, 64, 37, 49, 67, 16, 78, 59, 35, 32, 20, 63, 85, 15, 74, 83,
        95, 71, 38, 23, 93, 25, 61, 90, 45, 31, 26, 68, 12, 29,  9, 40,
         5, 89,  6, 48, 87, 80, 88, 65, 60, 24, 27, 10, 19, 96, 51,  2,
         0, 21, 56, 53],
       [27, 26, 36, 31, 10, 57, 30, 51, 67, 66, 81, 22, 60, 93, 59, 87,
        41,  7,  0, 35, 70, 47, 74, 85,  2, 80, 12, 68, 96, 34, 33,  8,
        89, 65, 18, 61, 77, 56, 20, 14, 38, 42, 21, 43, 82,  3, 29, 53,
        28, 86, 55, 46, 44, 58, 90, 54, 49, 13, 39, 83, 94, 32, 24, 84,
        71, 75, 23, 37, 98, 48, 92, 40, 52, 16, 15,  4, 72, 62, 97, 99,
         6, 19, 95, 45, 63, 78,  9,  5, 50, 76,  1, 11, 17, 79, 88, 91,
        25, 69, 73, 64]])

In [101]:
%timeit join_solutions(init_pop(), init_pop(), 4)

478 μs ± 48.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Edge Assembly Crossover

In [102]:
l_a = 2
l_b = 3


@njit
def build_costs_mat(a, b):
    idxs = (
        np.repeat(a, len(b)).reshape(len(a), len(b)) * problem.shape[1]
        + np.repeat(b, len(a)).reshape(len(b), len(a)).T
    ).reshape(-1)

    return problem.reshape(-1)[idxs].reshape(len(a), len(b))


print(build_costs_mat(np.arange(l_a), np.arange(l_b)))


[[  0.           4.9766      65.54479501]
 [  9.36499057   0.         110.87932346]]


In [103]:
@njit
def join_two_subtours(ta, tb):
    # We need to find two adjacent nodes in ta and two adjacent nodes in tb such that they can form two a-b edges while minizing the cost
    # Let the edges in ta be (ta[i], ta[i+1]) and in tb be (tb[j], tb[j+1])
    # We want to replace them with (ta[i], tb[j]) and (ta[i+1], tb[j+1]) OR (ta[i], tb[j+1]) and (ta[i+1], tb[j])
    # The cost change is:
    # cost(ta[i], tb[j]) + cost(ta[i+1], tb[j+1]) - cost(ta[i], ta[i+1]) - cost(tb[j], tb[j+1])
    # OR
    # cost(ta[i], tb[j+1]) + cost(ta[i+1], tb[j]) - cost(ta[i], ta[i+1]) - cost(tb[j], tb[j+1])

    ta_next = np.roll(ta, -1)
    tb_next = np.roll(tb, -1)

    costs_ta_tb = build_costs_mat(ta, tb)
    costs_ta_next_tb_next = build_costs_mat(ta_next, tb_next)
    costs_ta_tb_next = build_costs_mat(ta, tb_next)
    costs_ta_next_tb = build_costs_mat(ta_next, tb)

    costs_break_ta = problem.reshape(-1)[ta * problem.shape[1] + ta_next]
    costs_break_tb = problem.reshape(-1)[tb * problem.shape[1] + tb_next]

    costs1 = (
        costs_ta_tb
        + costs_ta_next_tb_next
        - costs_break_ta[:, np.newaxis]
        - costs_break_tb[np.newaxis, :]
    )
    costs2 = (
        costs_ta_tb_next
        + costs_ta_next_tb
        - costs_break_ta[:, np.newaxis]
        - costs_break_tb[np.newaxis, :]
    )

    min1 = costs1.min()
    min2 = costs2.min()

    if min1 < min2:
        idx = costs1.argmin()
        ai = int(np.floor(idx / costs1.shape[1]))
        bi = int(idx % costs1.shape[1])
        # Edges to form: (ta[ai], tb[bi]) and (ta[ai+1], tb[bi+1])
        # Tour: ...ta[ai], tb[bi], tb[bi-1]...tb[bi+1], ta[ai+1]...
        # Roll ta to have ta[ai] at the end
        # Roll tb to have tb[bi] at the start
        # Concatenate: roll(ta, -ai-1), roll(tb, -bi)
        return np.concatenate((np.roll(ta, -ai - 1), np.roll(tb, -bi)))
    else:
        idx = costs2.argmin()
        ai = int(np.floor(idx / costs2.shape[1]))
        bi = int(idx % costs2.shape[1])
        # Edges to form: (ta[ai], tb[bi+1]) and (ta[ai+1], tb[bi])
        # Tour: ...ta[ai], tb[bi+1], tb[bi+2]...tb[bi], ta[ai+1]...
        # Roll ta to have ta[ai] at the end
        # Roll tb and flip it to have tb[bi+1] at start and tb[bi] at end
        # Concatenate: roll(ta, -ai-1), roll(flip(tb), -bi-1)
        return np.concatenate((np.roll(ta, -ai - 1), np.roll(np.flip(tb), -bi - 1)))


@njit
def join_subtours(subtours: list[np.ndarray[np.int64]]):
    subtours = list(subtours)
    order = np.random.permutation(len(subtours))
    # subtours = sorted(subtours, key=lambda tour: len(tour))

    out = subtours[order[0]]

    for i in range(1, len(subtours)):
        out = join_two_subtours(out, subtours[order[i]])

    return out

In [104]:
TEST_SUBTOURS = [
    [
        44,
        91,
        78,
        26,
        96,
        89,
        43,
        5,
        57,
        39,
        10,
        1,
        71,
        79,
        17,
        65,
        84,
        24,
        20,
        8,
        87,
        80,
        92,
        33,
        3,
        18,
        76,
        60,
        19,
        63,
        46,
        47,
        77,
        53,
        40,
        22,
        90,
        99,
        73,
        55,
        67,
        50,
        74,
        68,
        27,
        0,
        93,
        49,
        6,
        9,
        37,
        86,
        85,
        70,
        62,
        72,
        59,
        42,
        58,
        2,
        7,
        34,
        64,
        69,
        54,
        28,
        41,
        88,
        15,
        32,
        12,
        66,
        30,
        61,
        21,
        98,
        94,
        48,
        52,
        56,
        23,
        25,
        45,
    ],
    [11, 51, 36, 81, 4],
    [75, 83, 31, 97, 95, 29],
    [13, 14, 38, 35, 82, 16],
]

TEST_SUBTOURS = [np.array(tour) for tour in TEST_SUBTOURS]

out = join_subtours(TEST_SUBTOURS)
out

array([67, 55, 73, 99, 90, 22, 40, 53, 77, 47, 46, 63, 19, 60, 76, 18,  3,
       33, 92, 80, 87,  8, 20, 24, 84, 65, 17, 79, 71,  1, 10, 39, 57,  5,
       43, 89, 96, 26, 78, 91, 44, 45, 25, 23, 56, 52, 48, 94, 98, 21, 61,
       30, 66, 12, 32, 15, 88, 41, 28, 54, 69, 64, 34,  7,  2, 58, 42, 59,
       72, 62, 70, 31, 97, 95, 29, 11, 51, 36, 81,  4, 75, 83, 85, 86, 37,
        9,  6, 49, 93,  0, 27, 68, 74, 50, 14, 38, 35, 82, 16, 13])

In [105]:
from time import perf_counter

In [106]:
from numba.types import UniTuple


@njit(UniTuple(nb.int64, 2)(nb.int64, nb.int64))
def make_edge(a: int, b: int) -> tuple[int, int]:
    return (nb.int64(a), nb.int64(b)) if a < b else (nb.int64(b), nb.int64(a))


@njit(UniTuple(nb.int64, 2)(UniTuple(nb.int64, 2)))
def sort_edge(edge: tuple[int, int]) -> tuple[int, int]:
    return make_edge(edge[0], edge[1])


edge_type = nb.types.UniTuple(nb.int64, 2)

In [107]:
@njit
def test_func():
    s = set()
    e1 = make_edge(2, 4)
    e2 = make_edge(4, 2)
    s.add(e1)
    s.add(e2)
    return len(s)


test_func()

1

In [108]:
# from typing import Iterable


@nb.experimental.jitclass(
    {
        "_s": nb.types.DictType(edge_type, nb.int64),
        "_l": nb.types.ListType(edge_type),
        "_adj_idx": nb.types.ListType(nb.types.ListType(nb.int64)),
    }
)
class EdgeSetAdv:
    # Advanced set of edges with adjacency index for fast lookup and O(1) random item selection

    def __init__(self, edges: list[tuple[int, int]]):
        self._l = nb.typed.List.empty_list(edge_type)
        for edge in edges:
            self._l.append(sort_edge(edge))

        # [(nb.int64(edge[0]), nb.int64(edge[1])) for edge in edges]
        self._s = nb.typed.Dict.empty(key_type=edge_type, value_type=nb.int64)
        # self._s = {}
        # for i, edge in enumerate(self._l):
        #     self._s[edge] = nb.int64(i)
        self._s = {edge: i for i, edge in enumerate(self._l)}
        # self._s
        self._adj_idx: list[list[int]] = nb.typed.List(
            [nb.typed.List.empty_list(nb.int64) for _ in range(ITEMS_NUM)]
        )
        for edge in edges:
            self._add_adj(edge)

    def _add_adj(self, edge: tuple[int, int]):
        self._adj_idx[edge[0]].append(edge[1])
        self._adj_idx[edge[1]].append(edge[0])

    def add(self, edge: tuple[int, int]):
        # print(f"Adding edge {edge}")
        edge = sort_edge(edge)
        self._l.append(edge)
        self._s[edge] = len(self._l) - 1
        self._add_adj(edge)

    def _remove_adj(self, edge: tuple[int, int]):
        self._adj_idx[edge[0]].remove(edge[1])
        self._adj_idx[edge[1]].remove(edge[0])

    def remove(self, edge: tuple[int, int]):
        # print(f"Removing edge {edge}")
        edge = sort_edge(edge)
        pos: int = self._s[edge]
        del self._s[edge]
        last: tuple[int, int] = self._l.pop()

        if pos < len(self._l):
            self._l[pos] = last
            self._s[last] = pos
        self._remove_adj(edge)

    def get_random(self) -> tuple[int, int]:
        idx = np.random.randint(len(self._l))
        return self._l[idx]

    def __contains__(self, edge: tuple[int, int]) -> bool:
        return sort_edge(edge) in self._s

    def get_adjacent(self, vertex: int) -> list[int]:
        out = list(self._adj_idx[vertex])
        # print(f"Adjacents of {vertex}: {out}")
        return out

    def get_set(self) -> set[tuple[int, int]]:
        return set(self._s.keys())

    def __str__(self):
        return str(self._s)

    # def __repr__(self):
    #     return repr(self._s)

    def __len__(self):
        return len(self._l)

    def empty(self) -> bool:
        return len(self._l) == 0

    def __sub__(self, other):
        out = EdgeSetAdv(list(self.get_set() - other.get_set()))
        return out

    def union(self, other):
        out = EdgeSetAdv(list(self.get_set().union(other.get_set())))
        return out


edge_set_adv_type = EdgeSetAdv.class_type.instance_type

In [109]:
ls = nb.typed.List(
    [
        (52, 99),
        (54, 99),
        (54, 95),
        (66, 95),
        (60, 66),
        (5, 60),
        (5, 62),
        (36, 62),
        (36, 55),
        (55, 57),
        (57, 74),
        (74, 78),
        (78, 92),
        (64, 92),
        (51, 64),
        (51, 89),
        (59, 89),
        (48, 59),
        (1, 48),
        (1, 80),
        (70, 80),
        (9, 70),
        (9, 21),
        (21, 34),
        (34, 84),
        (58, 84),
        (26, 58),
        (17, 26),
        (13, 17),
        (11, 13),
        (11, 68),
        (30, 68),
        (30, 85),
        (65, 85),
        (65, 97),
        (33, 97),
        (33, 61),
        (61, 83),
        (10, 83),
        (10, 32),
        (32, 37),
        (37, 42),
        (42, 77),
        (77, 98),
        (3, 98),
        (3, 88),
        (20, 88),
        (20, 94),
        (18, 94),
        (18, 27),
        (19, 27),
        (19, 69),
        (69, 73),
        (44, 73),
        (35, 44),
        (35, 75),
        (0, 75),
        (0, 93),
        (41, 93),
        (6, 41),
        (6, 29),
        (23, 29),
        (23, 45),
        (15, 45),
        (15, 82),
        (24, 82),
        (24, 28),
        (7, 28),
        (7, 76),
        (53, 76),
        (22, 53),
        (22, 72),
        (72, 79),
        (12, 79),
        (12, 90),
        (87, 90),
        (31, 87),
        (31, 71),
        (8, 71),
        (8, 67),
        (38, 67),
        (14, 38),
        (14, 81),
        (56, 81),
        (2, 56),
        (2, 43),
        (40, 43),
        (25, 40),
        (25, 49),
        (39, 49),
        (4, 39),
        (4, 91),
        (46, 91),
        (16, 46),
        (16, 47),
        (47, 96),
        (86, 96),
        (63, 86),
        (50, 63),
        (50, 52),
    ]
)
ls2 = nb.typed.List(
    [
        (8, 55),
        (8, 25),
        (25, 67),
        (20, 67),
        (20, 80),
        (49, 80),
        (22, 49),
        (22, 34),
        (34, 96),
        (15, 96),
        (13, 15),
        (13, 98),
        (48, 98),
        (3, 48),
        (3, 73),
        (73, 94),
        (79, 94),
        (28, 79),
        (28, 78),
        (78, 95),
        (62, 95),
        (62, 87),
        (36, 87),
        (10, 36),
        (10, 58),
        (58, 89),
        (2, 89),
        (2, 32),
        (32, 37),
        (37, 99),
        (19, 99),
        (19, 50),
        (26, 50),
        (6, 26),
        (6, 93),
        (81, 93),
        (44, 81),
        (16, 44),
        (16, 59),
        (59, 63),
        (47, 63),
        (47, 57),
        (56, 57),
        (9, 56),
        (9, 30),
        (30, 54),
        (27, 54),
        (14, 27),
        (11, 14),
        (11, 61),
        (61, 72),
        (72, 83),
        (83, 92),
        (7, 92),
        (7, 43),
        (40, 43),
        (40, 75),
        (75, 86),
        (60, 86),
        (60, 68),
        (68, 91),
        (77, 91),
        (4, 77),
        (1, 4),
        (1, 24),
        (24, 65),
        (45, 65),
        (45, 46),
        (46, 71),
        (38, 71),
        (38, 66),
        (33, 66),
        (33, 51),
        (21, 51),
        (12, 21),
        (12, 74),
        (69, 74),
        (39, 69),
        (29, 39),
        (17, 29),
        (5, 17),
        (5, 70),
        (18, 70),
        (18, 82),
        (42, 82),
        (23, 42),
        (23, 52),
        (31, 52),
        (31, 53),
        (41, 53),
        (41, 76),
        (76, 90),
        (88, 90),
        (0, 88),
        (0, 35),
        (35, 85),
        (64, 85),
        (64, 97),
        (84, 97),
        (55, 84),
    ]
)


es = EdgeSetAdv(ls)
# es.add(make_edge(3, 4))
# es.remove(make_edge(1, 2))

In [110]:
@nb.experimental.jitclass(
    {
        "starting_parent": nb.types.int64,
        "vertices": nb.types.ListType(nb.types.int64),
        "edges_a": nb.types.ListType(edge_type),
        "edges_b": nb.types.ListType(edge_type),
    }
)
class ABCycle:
    starting_parent: int  # 0 for A, 1 for B
    vertices: list[int]
    edges_a: list[tuple[int, int]]
    edges_b: list[tuple[int, int]]

    def __init__(
        self,
        starting_parent: int,
        vertices: list[int],
        edges_a: list[tuple[int, int]],
        edges_b: list[tuple[int, int]],
    ):
        self.starting_parent = starting_parent
        self.vertices = vertices
        self.edges_a = edges_a
        self.edges_b = edges_b


ab_cycle_type = ABCycle.class_type.instance_type


In [111]:
@njit(nb.types.ListType(edge_type)(nb.types.Array(nb.int64, 1, "A", readonly=True)))
def vertices_to_edges(vertices: np.ndarray) -> list[tuple[int, int]]:
    out = nb.typed.List()
    for i in range(0, len(vertices) - 1):
        out.append(make_edge(vertices[i], vertices[i + 1]))
    out.append(make_edge(vertices[0], vertices[-1]))
    return out


@njit(ab_cycle_type(edge_set_adv_type, edge_set_adv_type))
def init_new_cycle(edges_left_a: EdgeSetAdv, edges_left_b: EdgeSetAdv) -> ABCycle:
    starting_parent = random.getrandbits(1)
    edges_left = edges_left_a if starting_parent == 0 else edges_left_b
    new_first_edge = edges_left.get_random()
    edges_left.remove(new_first_edge)

    cycle = ABCycle(
        starting_parent,
        nb.typed.List([new_first_edge[0], new_first_edge[1]]),
        nb.typed.List.empty_list(edge_type),
        nb.typed.List.empty_list(edge_type),
    )

    if random.getrandbits(1):
        cycle.vertices.reverse()

    edges_used = cycle.edges_a if starting_parent == 0 else cycle.edges_b
    edges_used.append(new_first_edge)

    return cycle


@njit(
    nb.types.ListType(ab_cycle_type)(
        nb.types.ListType(edge_type), nb.types.ListType(edge_type)
    )
)
def partition_ab_cycles(
    edges_a: list[tuple[int, int]], edges_b: list[tuple[int, int]]
) -> list[ABCycle]:
    # Partitions the edges of parents A and B into AB-cycles

    time_a = 0  # 9% of time

    # print(edges_a, edges_b)

    edges_left_a = EdgeSetAdv(edges_a)
    edges_left_b = EdgeSetAdv(edges_b)

    time_b = 0

    cycles = nb.typed.List([init_new_cycle(edges_left_a, edges_left_b)])
    current_parent = 1 - cycles[-1].starting_parent

    time_c = 0
    time_d = 0
    time_e3 = 0
    time_f = 0

    while True:
        ab_cycle = cycles[-1]

        edges_left, edges_used = (
            (edges_left_a, ab_cycle.edges_a)
            if current_parent == 0
            else (edges_left_b, ab_cycle.edges_b)
        )

        time_c_start = 0  # 18% of time

        adj = edges_left.get_adjacent(ab_cycle.vertices[-1])

        if current_parent == ab_cycle.starting_parent:
            # We cannot close the cycle yet
            adj = [v for v in adj if v != ab_cycle.vertices[0]]
        else:
            if len(ab_cycle.vertices) > 2 and ab_cycle.vertices[0] in adj:
                # We can close the cycle right now
                adj = [ab_cycle.vertices[0]]

        time_c += 0 - time_c_start

        if not adj:
            time_d_start = 0  # 5% of time
            # Let's scrap this last cycle and start a new one

            # Add back the used edges
            for edge in ab_cycle.edges_a:
                edges_left_a.add(edge)
            for edge in ab_cycle.edges_b:
                edges_left_b.add(edge)
            # Init a new cycle
            cycles[-1] = init_new_cycle(edges_left_a, edges_left_b)
            current_parent = 1 - cycles[-1].starting_parent
            time_d += 0 - time_d_start

            continue

        new_vertex = adj[0 if len(adj) == 1 else bool(random.getrandbits(1))]

        time_e3_start = 0  # 47% of time
        # new_edge = make_edge(ab_cycle.vertices[-1], new_vertex)
        edge = (ab_cycle.vertices[-1], new_vertex)
        new_edge = sort_edge(edge)
        edges_left.remove(new_edge)
        edges_used.append(new_edge)
        time_e3 += 0 - time_e3_start

        if edges_left_a.empty() and edges_left_b.empty():
            break

        # raise NotImplementedError()
        if new_vertex == ab_cycle.vertices[0]:
            # Cycle completed, start a new one
            time_f_start = 0  # 21% of time

            new_cycle = init_new_cycle(edges_left_a, edges_left_b)
            cycles.append(new_cycle)
            current_parent = 1 - new_cycle.starting_parent
            time_f += 0 - time_f_start
            continue
        else:
            ab_cycle.vertices.append(new_vertex)

        current_parent = 1 - current_parent

    return cycles

In [112]:
population = init_pop()

edges_a = vertices_to_edges(population[rng.integers(0, PARAMS.pop_size)])
edges_b = vertices_to_edges(population[rng.integers(0, PARAMS.pop_size)])

partition_ab_cycles(edges_a, edges_b)

ListType[instance.jitclass.ABCycle#191fdda2be0<starting_parent:int64,vertices:ListType[int64],edges_a:ListType[UniTuple(int64 x 2)],edges_b:ListType[UniTuple(int64 x 2)]>]([<numba.experimental.jitclass.boxing.ABCycle object at 0x00000191890BCD60>, <numba.experimental.jitclass.boxing.ABCycle object at 0x000001918941AF50>, <numba.experimental.jitclass.boxing.ABCycle object at 0x000001918EA08D60>, <numba.experimental.jitclass.boxing.ABCycle object at 0x000001918FD72890>])

In [113]:
# https://ir.lib.nycu.edu.tw/bitstream/11536/27818/1/000183911700005.pdf


@njit(
    nb.types.Array(nb.int64, 1, "C")(
        nb.types.Array(nb.int64, 1, "A"), nb.types.Array(nb.int64, 1, "A")
    )
)
def eax(p_a: np.ndarray, p_b: np.ndarray) -> np.ndarray:
    # Performs edge assembly crossover between parents A and B
    # and returns a single offspring solution

    L = p_a.shape[0]

    # edges_a = vertices_to_edge_set(p_a)
    # edges_b = vertices_to_edge_set(p_b)

    # time_a = perf_counter()

    # Partition the multigraph formed by edges of parents A and B into AB-cycles
    edges_a = vertices_to_edges(p_a)
    edges_y = set(edges_a)
    edges_b = vertices_to_edges(p_b)

    # time_b = perf_counter()
    # b to c: 48%

    ab_cycles = partition_ab_cycles(edges_a, edges_b)
    # return p_a

    # time_c = perf_counter()

    # Filter out degenerate cycles (with only 2 edges)
    # ab_cycles = nb.typed.List.empty_list(ab_cycle_type)
    # for cycle in partition_ab_cycles(edges_a, edges_b):
    #     if len(cycle.edges_a) > 1:
    #         ab_cycles.append(cycle)

    # Randomly select a subset of AB-cycles to use for crossover
    # use_n = np.random.randint(1, int(np.ceil(len(ab_cycles) / 2)) + 1)
    use_n = 1

    # return p_a

    while True:
        if len(ab_cycles) == 0:
            return eax(p_a, p_b)

        ab_cycles_id = np.random.randint(0, len(ab_cycles))
        ab_cycle = ab_cycles[ab_cycles_id]
        # Remove edges (only from parent A)

        # tmp_edges_y = edges_y.copy()

        for edge in ab_cycle.edges_a:
            edges_y.remove(edge)

        # Add edges (only from parent B)
        for edge in ab_cycle.edges_b:
            edges_y.add(edge)

        if len(edges_y) != L:
            edges_y = set(vertices_to_edges(p_a))
            ab_cycles.pop(ab_cycles_id)
            continue
        else:
            break

    edges_y = nb.typed.List(list(edges_y))

    # Reconstruct subtours from the edge set
    visited = np.zeros((L), dtype=np.bool)
    subtours = nb.typed.List([nb.typed.List.empty_list(nb.int64)])

    while len(edges_y) > 0:
        curr_subtour = subtours[-1]

        if len(curr_subtour) == 0:
            edge = edges_y.pop()
            curr_subtour.append(edge[0])
            curr_subtour.append(edge[1])
            visited[edge[0]] = True
            visited[edge[1]] = True
        else:
            found = False
            for edge in edges_y:
                if edge[0] == curr_subtour[-1] and not visited[edge[1]]:
                    edges_y.remove(edge)
                    curr_subtour.append(edge[1])
                    visited[edge[1]] = True
                    found = True
                    break

                if edge[1] == curr_subtour[-1] and not visited[edge[0]]:
                    edges_y.remove(edge)
                    curr_subtour.append(edge[0])
                    visited[edge[0]] = True
                    found = True
                    break

            if not found:
                edge = make_edge(curr_subtour[-1], curr_subtour[0])

                if edge in edges_y:
                    edges_y.remove(edge)
                if len(edges_y) > 0:
                    subtours.append(nb.typed.List.empty_list(nb.int64))

    # Join the subtours into a single tour
    out = join_subtours([np.array(list(t)) for t in subtours])

    return out

In [114]:
eax(population[3], population[1])

array([95, 75,  8, 54, 85, 57, 27, 43, 73, 61, 26, 18, 62, 53, 67, 36, 71,
       69, 93,  0, 79, 31, 94, 38, 68, 56, 40, 97, 84, 28, 52, 41, 87, 83,
        7, 63, 33, 11, 66, 35, 10,  5, 47, 46, 91, 78, 58, 16,  9,  3, 51,
       55, 20, 29, 22, 60, 37, 25, 14, 39, 17, 90, 21, 64,  2, 98, 86, 59,
        4,  6, 49, 77, 96, 42, 80, 13, 34, 99, 81, 76, 30, 70, 48, 44, 19,
       65, 72, 24, 45, 23, 12,  1, 32, 88, 15, 82, 74, 50, 89, 92])

In [115]:
# %timeit eax(population[rng.integers(0, PARAMS.pop_size)], population[rng.integers(0, PARAMS.pop_size)])

In [116]:
@nb.njit(
    nb.types.Array(nb.int64, 2, "C")(
        nb.types.Array(nb.int64, 2, "A"), nb.types.Array(nb.int64, 2, "A"), nb.int64
    ),
    parallel=True,
)
def eax_loop(
    parents_a: np.ndarray, parents_b: np.ndarray, children_per_pair: int
) -> np.ndarray:
    # print("EAX loop called")

    N, L = parents_a.shape

    out = np.empty((N, L), dtype=np.int64)
    # time_total = np.zeros((5), dtype=np.float64)

    for i in nb.prange(N):
        best_fitness = np.inf
        for j in nb.prange(children_per_pair):
            child = eax(parents_a[i], parents_b[i])
            child_fitness = calc_pop_fitness(child[np.newaxis, :])[0]
            if child_fitness < best_fitness:
                best_fitness = child_fitness
                out[i] = child
        # print(
        #     f"EAX on individual {i + 1}/{N} between parents {parents_a[i]} and {parents_b[i]}"
        # )
        # out[i] = eax(parents_a[i], parents_b[i])
        # time_total += time

    return out

In [117]:
population[1]

array([17,  5, 21, 85, 43, 39, 97, 96, 12, 84,  3,  0, 11, 87, 30, 51, 20,
       36, 83, 34, 54, 26,  7, 27,  2, 80,  1, 60, 28, 62,  8, 23, 73, 24,
       98, 74, 82, 81, 65, 72, 14, 15, 44, 86,  9, 68, 93, 10, 22, 67, 95,
       75, 58, 99, 32, 57, 45, 42, 50, 29, 49, 79, 56, 64, 59, 94, 47, 19,
       16, 46, 25,  6, 71, 52, 63, 55, 77, 89, 33, 91, 53, 90, 31, 48, 37,
       40, 66, 78, 92, 70, 41, 88, 18,  4, 13, 61, 69, 76, 35, 38])

In [118]:
population = init_pop()

eax_loop(population[[1, 2, 3]], population[[2, 3, 1]], 2)

array([[ 2, 36, 58, 93, 66,  5, 91, 61, 67, 38, 59, 29, 31, 28, 47, 41,
        94, 81, 35, 87, 37, 21, 49, 33, 14, 25, 80, 65, 74, 92, 42, 88,
        17, 23, 85, 44, 19, 71, 76, 54, 39,  6, 69, 73, 15, 78, 62, 95,
        10, 55, 60, 86,  9, 43, 16, 84, 48, 12, 13, 64, 30, 26, 63, 99,
        34, 20, 27, 45, 56, 22, 77, 11, 82, 90,  3,  1, 18,  8, 75, 53,
        46, 51, 96, 24,  0, 83, 79, 70, 68,  4, 98, 72,  7, 52, 97, 89,
        32, 40, 50, 57],
       [27, 45, 56, 46, 51, 84, 10, 21, 20, 52, 91, 40, 32,  4, 92, 60,
        86, 42, 90, 87, 73, 79, 75, 16, 43,  9, 65, 74, 48, 71, 76, 97,
        55, 93, 22, 70, 29, 31, 83, 54, 38, 18, 47, 41, 34, 62, 95, 81,
        94, 35, 77, 50, 49, 82, 15,  3, 12, 66,  5, 61, 23, 19, 44, 11,
        28,  8, 26,  6, 57, 88, 67, 69, 37, 68, 53, 63, 39, 85,  1, 24,
         2, 33, 72,  7, 36, 96, 30, 99, 89, 25, 13, 59, 64,  0, 78, 98,
        58, 80, 17, 14],
       [20, 19, 88, 57,  4, 52, 76, 15, 38, 18, 51, 62, 34, 23, 25, 13,
        12, 66

In [119]:
# %timeit eax_loop(init_pop(), init_pop(), 4)

For 1000 items problems, the eax crossover is around 451x times slower, despite the optimization efforts. This makes the eax crossover impractical for use in these problems, and thus was not selected for the final implementation.

### Selection mechanisms

In [120]:
@functools.lru_cache(maxsize=2)
def calc_exp_dist(pop_size, w):
    probs = np.pow(w, np.arange(pop_size - 1, -1, -1))
    probs /= probs.sum()
    return probs


def selection_exp_probs(fitness_rank, to_select, w):
    N = fitness_rank.shape[0]
    probs = calc_exp_dist(N, w)

    probs = probs[fitness_rank]

    return rng.choice(np.arange(N), (to_select), p=probs, replace=False)


@functools.lru_cache(maxsize=2)
def calc_sig_dist(pop_size, s):
    probs = 1 / (1 + np.exp(-2 * s * np.arange(pop_size) / pop_size + s))
    probs /= probs.sum()
    return probs


def selection_sig_probs(fitness_rank, to_select, s=5):
    N = fitness_rank.shape[0]
    probs = calc_sig_dist(N, s)

    return rng.choice(fitness_rank, (to_select), p=probs, replace=False)


def selection_trunc(fitness_rank, to_select):
    N = fitness_rank.shape[0]
    parents_ids = fitness_rank[N - to_select :]
    return parents_ids

In [121]:
selection_trunc(np.arange(PARAMS.pop_size), 10)

array([40, 41, 42, 43, 44, 45, 46, 47, 48, 49])

In [122]:
selection_sig_probs(np.arange(PARAMS.pop_size), 10, s=10)

array([42, 22, 39,  9, 47, 44, 45, 36, 31, 41])

## Evolution cycle

In [123]:
# children_to_mutate = int(
#     PARAMS.pop_size * PARAMS.churn_rate * PARAMS.mutate_children_rate
# )


def get_w(temp):
    return 0.8 + (1 - temp) * 0.15


def get_s(temp):
    return 9 + (1 - temp) * 5


assert PARAMS.pop_size % PARAMS.num_islands == 0
ISLAND_POP_SIZE = PARAMS.pop_size // PARAMS.num_islands

to_churn = int(PARAMS.pop_size / PARAMS.num_islands * PARAMS.churn_rate)


def evolve_island(mutate_func, island_i, temp, i):
    # Evolve a single island

    pop = population[island_i * ISLAND_POP_SIZE : (island_i + 1) * ISLAND_POP_SIZE]

    # Calculate fitness and rank
    fitness = calc_pop_fitness(pop)
    fitness_rank = (-fitness).argsort()

    # Select parents
    w = get_w(temp)
    parent_ids = selection_exp_probs(fitness_rank, to_churn, w)
    # parent_ids = selection_trunc(fitness_rank, to_churn)
    # parent_ids = np.arange(PARAMS.pop_size)
    parent_ids2 = rng.permutation(parent_ids)

    # offspring = eax_loop(
    #     pop[parent_ids],
    #     pop[parent_ids2],
    #     children_per_pair=PARAMS.children_per_pair,
    # )
    offspring = join_solutions(
        pop[parent_ids],
        pop[parent_ids2],
        children_per_pair=PARAMS.children_per_pair,
    )

    # Mutate some or all offspring
    offspring = mutate_func(offspring)

    # children_to_mutate = int(offspring.shape[0] * PARAMS.mutate_children_rate * temp)
    # if children_to_mutate > 0:
    #     mutate_ids = rng.choice(
    #         np.arange(offspring.shape[0]), (children_to_mutate), replace=False
    #     )
    #     offspring[mutate_ids] = mutate_func(offspring[mutate_ids])

    offspring_fitness = calc_pop_fitness(offspring)
    fitness = np.concatenate((fitness, offspring_fitness), axis=0)
    fitness_rank = (-fitness).argsort()

    s = get_s(temp)
    keep_ids = selection_sig_probs(fitness_rank, ISLAND_POP_SIZE, s)
    # keep_ids = selection_exp_probs(fitness_rank, PARAMS.pop_size, w)
    # keep_ids = selection_trunc(fitness_rank, ISLAND_POP_SIZE)

    keep_old_ids = keep_ids[keep_ids < ISLAND_POP_SIZE]
    keep_new_ids = keep_ids[keep_ids >= ISLAND_POP_SIZE] - ISLAND_POP_SIZE
    remove_old_ids = np.ones((ISLAND_POP_SIZE), dtype=np.bool)
    remove_old_ids[keep_old_ids] = 0
    (remove_old_ids,) = np.where(remove_old_ids)

    pop[remove_old_ids] = offspring[keep_new_ids]


def evolution_cycle(mutate_func, temp=0.8, i=0):
    # population[:, :] = mutate_func(population)

    if PARAMS.wipeout_every > 0 and i > 0 and i % PARAMS.wipeout_every == 0:
        print("Wipeout event!")
        # print(population)
        fitness = calc_pop_fitness(population)
        fitness_rank = (-fitness).argsort()
        # survivor_ids = selection_exp_probs(fitness_rank, PARAMS.survivors_num, get_w(0))
        survivor_ids = selection_trunc(fitness_rank, PARAMS.survivors_num)
        new_population = init_pop()
        new_population[: PARAMS.survivors_num] = population[survivor_ids]
        population[:] = new_population
        # print("Population after wipeout:")
        # print(population)

    # population wipeout
    # island model: two pops distinct, no interaction -> then swap some individuals (migrants) at some time to mix

    for island_i in range(PARAMS.num_islands):
        evolve_island(mutate_func, island_i, temp, i)

    if (
        PARAMS.migration_every > 0
        and PARAMS.num_islands > 1
        and i > 0
        and i % PARAMS.migration_every == 0
    ):
        # print("Migration event!")
        # Migrate some individuals between islands
        migrants_per_island = int(ISLAND_POP_SIZE * PARAMS.migration_rate)
        island_order = np.random.permutation(PARAMS.num_islands)

        for island_i in island_order:
            source_island_i = island_i
            target_island_i = (island_i + 1) % PARAMS.num_islands

            source_start = source_island_i * ISLAND_POP_SIZE
            source_end = (source_island_i + 1) * ISLAND_POP_SIZE
            target_start = target_island_i * ISLAND_POP_SIZE
            target_end = (target_island_i + 1) * ISLAND_POP_SIZE

            source_pop = population[source_start:source_end]
            target_pop = population[target_start:target_end]

            # Calculate fitness and rank in source island
            source_fitness = calc_pop_fitness(source_pop)
            source_fitness_rank = (-source_fitness).argsort()
            target_fitness = calc_pop_fitness(target_pop)
            target_fitness_rank = (-target_fitness).argsort()

            # Select migrants
            # migrant_ids = selection_trunc(fitness_rank, migrants_per_island)
            source_migrant_ids = selection_exp_probs(
                source_fitness_rank, migrants_per_island, get_w(temp)
            )
            target_migrant_ids = selection_exp_probs(
                target_fitness_rank, migrants_per_island, get_w(temp)
            )

            target_migrants = target_pop[target_migrant_ids].copy()
            target_pop[target_migrant_ids] = source_pop[source_migrant_ids]
            source_pop[source_migrant_ids] = target_migrants

    fitness = calc_pop_fitness(population)
    best_sol_id = fitness.argmin()

    return population[best_sol_id], fitness[best_sol_id]


In [124]:
TEMP_CHANGE_RATE = 3_000


def temp_schedule(i):
    if i < TEMP_CHANGE_RATE:
        return 1

    if i < TEMP_CHANGE_RATE * 6:
        return (TEMP_CHANGE_RATE * 6 - i) / (TEMP_CHANGE_RATE * 5) / 2 + 0.5

    return (np.sin((i - TEMP_CHANGE_RATE * 6) / TEMP_CHANGE_RATE) + 1) / 2


# x = np.arange(250_000)
# plt.plot(x, [temp_schedule(v) for v in x])

In [125]:
# %timeit evolution_cycle(mutate_random, temp=0.8)

In [126]:
# %timeit evolution_cycle(mutate_solutions_swap, temp=0.8, i=0)

In [ ]:
# population = init_pop()

MAX_ITER = 5_000_000
REPORT_EVERY = 100

for i in range(MAX_ITER):
    t = temp_schedule(i)
    best_sol, min_fitness = evolution_cycle(mutate_random, 0, i)

    if i % REPORT_EVERY == 0 and i != 0:
        print(f"Max fitness: {min_fitness:.2f} - Temp: {t:.2f}")


# Results

- problem_g_100 - found 4000.84
- problem_r1_100 - found 715.72
- problem_r2_100 - found -4787.97

--

- problem_g_500 - found 9420.07
- problem_r1_500 - found 2457.90
- problem_r2_500 - found -22376.36

--

- problem_g_1000 - found 17221.00
- problem_r1_1000 - found 11678.28
- problem_r2_1000 - found -46227.20



